In [3]:
# Importamos las librerías necesarias
import pandas as pd # type: ignore
import numpy as np # type: ignore
import psycopg2 as ps # type: ignore

In [4]:
# Creamos la conexión a la base de datos
# Vamos a crear una conexión a la base de datos.
conn = ps.connect(
    dbname = "BBDD_Hoteles", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [5]:
# Creamos un cursor el cual nos va a permitir ejecutar querys.
cur = conn.cursor()

In [6]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

## Bonus Track 2

## Consulta 1: Cuantos hoteles tiene la base de datos


In [13]:
query1 = """
    SELECT 
        count(id_hotel) 
    FROM hoteles; 
"""
cur.execute(query1)
q1 = cur.fetchall()
print(f"La base de datos tiene {q1[0][0]} hoteles")

La base de datos tiene 29 hoteles


## Consulta 2: Cuantas reservas se han hecho

In [15]:
query2 = """
    SELECT 
        count(id_reserva) 
    FROM reservas; 
"""
cur.execute(query2)
q2 = cur.fetchall()
print(f"Se han realizado un total de {q2[0][0]} reservas")

Se han realizado un total de 15000 reservas


## Consulta 3: Identifica los 10 clientes que más se han gastado


In [29]:
query3 = """
    SELECT 
        c.nombre,
        sum(r.precio_noche)
    FROM clientes as c
        INNER JOIN  reservas as r ON c.id_cliente = r.id_cliente
    GROUP BY c.nombre
    ORDER BY 2 DESC
    LIMIT 10; 
"""
cur.execute(query3)
q3 = cur.fetchall()
q3[1][0]
print(f"Los clientes que más se han gastado son: {q3[0][0]} con  {q3[0][1]}€, {q3[1][0]} con  {q3[1][1]}€, {q3[1][0]} con  {q3[0][1]}€, {q3[3][0]} con  {q3[0][1]}€, {q3[4][0]} con  {q3[5][0]}€, {q3[0][0]} con  {q3[0][1]}€, {q3[0][0]} con  {q3[0][1]}€, {q3[0][0]} con  {q3[0][1]}€, {q3[0][0]} con  {q3[0][1]}€, {q3[0][0]} con  {q3[0][1]}€, {q3[0][0]} con  {q3[0][1]}€. ")

Los clientes que más se han gastado son: Nazaret con  9194.233058437145€, Reyes con  8830.594966228322€, Reyes con  9194.233058437145€, Amor con  9194.233058437145€, Trinidad con  Cruz€, Nazaret con  9194.233058437145€, Nazaret con  9194.233058437145€, Nazaret con  9194.233058437145€, Nazaret con  9194.233058437145€, Nazaret con  9194.233058437145€, Nazaret con  9194.233058437145€. 


## Consulta 4: Identifica el hotel de la competencia y el hotel de nuestra marca que más han recaudado para esas fechas


In [56]:
conn.rollback()

In [31]:
# Hotel competencia con mas recaudacion
query4 = """ 
        SELECT 
            h.nombre_hotel,
            sum(r.precio_noche)
        FROM hoteles as h
            INNER JOIN  reservas as r ON h.id_hotel = r.id_hotel
        WHERE h.competencia = True
        GROUP BY h.nombre_hotel
        ORDER BY 2 DESC
        LIMIT 1; 
"""

cur.execute(query4)
q4 = cur.fetchall()
print(f"El hotel de la competencia que más ha recaudado ha sido {q4[0][0]}, con un total de {q4[0][1]}€.")

El hotel de la competencia que más ha recaudado ha sido ibis Madrid Centro las Ventas, con un total de 90128.0€.


In [32]:
# Hotel grupo con mas recaudacion
query4_1 = """ 
        SELECT 
            h.nombre_hotel,
            sum(r.precio_noche)
        FROM hoteles as h
            INNER JOIN  reservas as r ON h.id_hotel = r.id_hotel
        WHERE h.competencia = False
        GROUP BY h.nombre_hotel
        ORDER BY 2 DESC
        LIMIT 1; 
"""

cur.execute(query4_1)
q5 =cur.fetchall()
print(f"El hotel del grupo que más ha recaudado ha sido {q5[0][0]}, con un total de {q5[0][1]}€.")

El hotel del grupo que más ha recaudado ha sido Hotel Monte Verde, con un total de 152136.8095238094€.


## Consulta 5: Identifica cuantos eventos hay.


In [33]:
# Hotel competencia con mas recaudacion
query5 = """ 
        SELECT 
            count(id_evento)
        FROM eventos ; 
"""

cur.execute(query5)
q6 = cur.fetchall()
print(f"Hay un total de {q6[0][0]} eventos.")

Hay un total de 217 eventos.


## Consulta 6: Identifica el día que más reservas se han hecho para nuestro hoteles

In [39]:
# Hotel competencia con mas recaudacion
query6 = """ 
        SELECT 
            r.fecha_reserva,
            count(r.id_reserva)
        FROM hoteles as h
            INNER JOIN  reservas as r ON h.id_hotel = r.id_hotel
        WHERE h.competencia = True
        GROUP BY r.fecha_reserva ; 
"""

cur.execute(query6)
q7 = cur.fetchall()
print(f"El dia que mas reservas se han realizado ha sido el {q7[0][0]} con un total de {q7[0][1]} reservas.")


El dia que mas reservas se han realizado ha sido el 2025-02-21 con un total de 5172 reservas.


In [48]:
# una vez hemos terminado de trabajar es necesario cerrar la conexión y el cursor.
cur.close()
conn.close()

## Bonus Track 2
Una vez creada la base de datos haz una o varias consultas para crear un dataframe con los datos relevantes para hacer un análisis de precios entre la competencia y nuestros hoteles para esas fechas. Realiza un análisis temporal de las fechas de reserva

Me interesa saber las reservas de cada hotel, los precios de cada reserva, 

In [63]:
# df hoteles grupo 
query7 = """ 
        SELECT 
            h.id_hotel ,
            h.nombre_hotel,
            h.valoracion,
            r.fecha_reserva,
            r.precio_noche,
            c.id_cliente,
            concat(c."nombre", ' ', c."apellido")
        FROM hoteles as h
            INNER JOIN  reservas AS r ON h.id_hotel = r.id_hotel
            INNER JOIN  clientes AS c ON r.id_cliente = c.id_cliente
        WHERE h.competencia = False; 
"""

cur.execute(query7)
q7 = cur.fetchall()
df_grupo = pd.DataFrame(q7)
df_grupo = df_grupo.rename(columns={0: "id_hotel", 1: "nombre_hotel", 2: "valoracion", 3: "fecha_reserva", 4: "precio_noche", 5: "id_cliente", 6: "nombre_cliente"})
df_grupo

,id_hotel,nombre_hotel,valoracion,fecha_reserva,precio_noche,id_cliente,nombre_cliente
0,1,Hotel Monte Verde,3.10,2025-02-09,276.612381,13141,Maite Robles
1,2,Hotel Brisas del Mar,3.09,2025-02-08,275.225921,14814,Tecla Ferrán
2,3,Hotel Camino del Sol,3.07,2025-02-02,269.998444,13479,Amílcar Corominas
3,4,Hotel Puerta del Cielo,3.03,2025-02-08,280.151243,14680,Joan Plaza
4,5,Hotel Encanto Real,3.03,2025-02-09,278.243996,12680,Chelo Arévalo
...,...,...,...,...,...,...,...
9823,9,Gran Hotel Madrid,3.07,2025-02-07,264.967165,14755,Héctor Alsina
9824,16,Hotel Rincón Sereno,3.00,2025-02-06,277.318824,14551,Edgardo Puig
9825,7,Hotel Jardines del Rey,2.94,2025-02-06,272.402335,14346,Lorena Morell
9826,2,Hotel Brisas del Mar,3.09,2025-02-11,275.225921,14481,Nadia Aznar


In [62]:
# df hoteles competencia 
query8 = """ 
        SELECT 
            h.id_hotel ,
            h.nombre_hotel,
            h.valoracion,
            r.fecha_reserva,
            r.precio_noche,
            c.id_cliente,
            concat(c."nombre", ' ', c."apellido")
        FROM hoteles as h
            INNER JOIN  reservas as r ON h.id_hotel = r.id_hotel
            INNER JOIN  clientes AS c ON r.id_cliente = c.id_cliente
        WHERE h.competencia = True; 
"""

cur.execute(query8)
q8 = cur.fetchall()
df_competencia = pd.DataFrame(q8)
df_competencia = df_competencia.rename(columns={0: "id_hotel", 1: "nombre_hotel", 2: "valoracion", 3: "fecha_reserva", 4: "precio_noche", 5: "id_cliente", 6: "nombre_cliente"})
df_competencia

,id_hotel,nombre_hotel,valoracion,fecha_reserva,precio_noche,id_cliente,nombre_cliente
0,20,ibis Styles Madrid Prado,4.7,2025-02-21,161.0,14213,Feliciana Barberá
1,21,ibis budget Madrid Calle 30,4.4,2025-02-21,110.0,13282,Leonardo Aguiló
2,22,ibis Madrid Centro las Ventas,4.5,2025-02-21,172.0,14238,Judith Sáenz
3,23,ibis budget Madrid Centro las Ventas,4.3,2025-02-21,119.0,12657,Adolfo Menendez
4,24,ibis budget Madrid Vallecas,4.3,2025-02-21,102.0,14229,María Carmen Moliner
...,...,...,...,...,...,...,...
5167,28,ibis Madrid Alcobendas,4.4,2025-02-21,85.0,14901,Jesusa Espinosa
5168,26,ibis Madrid Alcorcon Tresaguas,4.4,2025-02-21,90.0,14902,Cruz Hurtado
5169,27,ibis budget Madrid Aeropuerto,4.0,2025-02-21,88.0,14903,Jose Francisco Cortes
5170,23,ibis budget Madrid Centro las Ventas,4.3,2025-02-21,119.0,14904,Toño Narváez
